<a href="https://colab.research.google.com/github/carywoods/50projects50days/blob/master/manual_BTA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

linear regression

In [ ]:
# Install the necessary libraries
# !pip install yfinance pandas matplotlib scikit-learn
#!pip install pandas==1.3.3 yfinance==0.1.63 numpy==1.21.2 matplotlib==3.4.3 scikit-learn==0.24.2
!pip install pandas==2.2.2 yfinance==0.1.63 numpy==1.21.2 matplotlib==3.4.3 scikit-learn==0.24.2
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Function to plot stock data
def plot_stock_analysis(ticker):
    # Fetch the stock data for the last 18 months
    end_date = pd.Timestamp.today()
    start_date = end_date - pd.DateOffset(months=18)
    data = yf.download(ticker, start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))

    # Calculate the 90-day moving average
    data['90_day_MA'] = data['Close'].rolling(window=90).mean()

    # Calculate the upper and lower control limits (mean +/- 2 standard deviations)
    data['Upper_Control_Limit'] = data['90_day_MA'] + 2 * data['Close'].rolling(window=90).std()
    data['Lower_Control_Limit'] = data['90_day_MA'] - 2 * data['Close'].rolling(window=90).std()

    # Ensure the Date column is the index and in datetime format
    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])

    # Linear Regression
    X = np.array((data['Date'] - data['Date'].min()).dt.days).reshape(-1, 1)
    y = data['Close']
    model = LinearRegression()
    model.fit(X, y)
    data['Trend'] = model.predict(X)

    # Determine if the trend line is positive or negative
    trend_slope = model.coef_[0]
    trend_direction = "Positive" if trend_slope > 0 else "Negative"

    # Identify breaches
    data['Signal'] = np.where(data['Close'] < data['Lower_Control_Limit'], 'Breach Lower',
                              np.where(data['Close'] > data['Upper_Control_Limit'], 'Breach Upper', 'Within Limits'))

    # Determine Buy, Hold, or Sell signals
    conditions = [
        (data['Close'] < data['Lower_Control_Limit']) & (trend_slope > 0),
        (trend_slope > 0) & (data['Close'] < data['Upper_Control_Limit']),
        (data['Close'] > data['Upper_Control_Limit']) & (trend_slope > 0)
    ]
    choices = ['Buy', 'Hold', 'Sell']
    data['Action'] = np.select(conditions, choices, default='Hold')

    # Determine the action for the latest data point
    latest_action = data.iloc[-1]['Action']
    latest_date = end_date.strftime('%Y-%m-%d')

    # Prepare the action annotation with today's date
    action_annotation = f"{latest_action}: {latest_date}"

    # Plot the data
    plt.figure(figsize=(16, 8))
    plt.plot(data['Date'], data['Close'], label='Close Price', color='blue')
    plt.plot(data['Date'], data['90_day_MA'], label='90-day MA', color='orange')
    plt.plot(data['Date'], data['Upper_Control_Limit'], label='Upper Control Limit', color='green')
    plt.plot(data['Date'], data['Lower_Control_Limit'], label='Lower Control Limit', color='red')
    plt.plot(data['Date'], data['Trend'], label='Trend Line', color='purple')

    # Create a combined legend
    legend_elements = [
        plt.Line2D([0], [0], color='blue', lw=2, label='Close Price'),
        plt.Line2D([0], [0], color='orange', lw=2, label='90-day MA'),
        plt.Line2D([0], [0], color='green', lw=2, label='Upper Control Limit'),
        plt.Line2D([0], [0], color='red', lw=2, label='Lower Control Limit'),
        plt.Line2D([0], [0], color='purple', lw=2, label='Trend Line'),
        plt.Line2D([0], [0], color='none', lw=0, label=action_annotation)
    ]

    plt.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
    plt.title(f'Stock Analysis for {ticker} - Trend Line: {trend_direction}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.show()

# Example usage
plot_stock_analysis("FTXH")


  Using cached yfinance-0.1.63.tar.gz (26 kB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.21.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached matplotlib-3.4.3.tar.gz (37.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached scikit-learn-0.24.2.tar.gz (7.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


[*********************100%***********************]  1 of 1 completed


ValueError: Cannot set a DataFrame with multiple columns to the single column Upper_Control_Limit